# 数值分析第一次大作业

参考书：《数值分析（第4版）》 颜庆津 北京航空航天大学出版社

题目：P239 第四题

In [1]:
import math
import numpy as np

In [2]:
k_dim = 1000
t_array =  [1 + 0.001 * (k - 1) for k in range(1, k_dim + 1)]
print(t_array[0])

1.0


In [3]:
def f_b(b, t):
    return math.exp(-t * b) - t * math.log(b)

使用割线法求解非线性方程（参考：P80）

In [4]:
def solve_b(k):
    iter_num = 0
    t = t_array[k - 1]
    x_ = 0
    x_k = 0.9
    x_k1 = 2
    while abs(x_k - x_) / abs(x_k) > 1e-12:
        iter_num += 1
        x_ = x_k
        x_k = x_k1
        x_k1 = x_k - (f_b(x_k, t) * (x_k - x_)) / (f_b(x_k, t) - f_b(x_, t)) # 割线法
        
#     print('Return after %d iterations' % iter_num)
    return x_k1

求b_k

In [5]:
b_array = [solve_b(k) for k in range(1, 1001)]
# print(b_array)

非线性方程组(A.1)

In [6]:
def F_A(x, k):
    # x为列向量，因此需二次索引
    x1 = x[0][0]
    x2 = x[1][0]
    x3 = x[2][0]
    x4 = x[3][0]
    x5 = x[4][0]
    t = t_array[k - 1]
    f1 = math.exp(-x1) + math.exp(-2 * x2) + x3 - 2 * x4 + t * x5 - 5.3
    f2 = math.exp(-2 * x1) + math.exp(-x2) - 2 * x3 + t * x4 - x5 + 25.6
    f3 = t * x1 + 3 * x2 + math.exp(-x3) - 3 * x5 + 37.8
    f4 = 2 * x1 + t * x2 + x3 - math.exp(-x4) + 2 * math.exp(-2 * x5) - 31.3
    f5 = x1 - 2 * x2 - 3 * t * x3 + math.exp(-2 * x4) + 3 * math.exp(-x5) + 42.1
    
    return np.array([[f1], [f2], [f3], [f4], [f5]])

In [7]:
def get_h(x, F, lower_bound):
    c = 2 
    h = c * np.linalg.norm(F) # 此处是牛顿-斯蒂芬森法，c1=c2=...=c5
    x = x + h * np.array([np.ones(5)]).T
    while sum(x >= lower_bound * np.array([np.ones(5)]).T) < 5:
        # 保证x + h在定义域内，否则继续增大h
        h = h * c
        x = x + h * np.array([np.ones(5)]).T
        
    return h * np.ones(5)

In [8]:
def J(x, h, k):
    J = np.zeros((5,5))
    e = np.eye(5)
    for i in range(5):
        J[:,[i]] = (F_A(x + h[i] * e[:,[i]], k) - F_A(x, k)) / h[i]
        
    return J

In [9]:
x = np.array([[3],[4],[5],[6],[7]])
h = [2,2,2,2,2]
print(J(x, h, 1))

[[-2.15245607e-02 -1.64659208e-04  1.00000000e+00 -2.00000000e+00
   1.00000000e+00]
 [-1.21667612e-03 -7.91844336e-03 -2.00000000e+00  1.00000000e+00
  -1.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00 -2.91303252e-03  0.00000000e+00
  -3.00000000e+00]
 [ 2.00000000e+00  1.00000000e+00  1.00000000e+00  1.07164477e-03
  -8.16298739e-07]
 [ 1.00000000e+00 -2.00000000e+00 -3.00000000e+00 -3.01583859e-06
  -1.18270824e-03]]


使用离散牛顿法求解非线性方程组（参考：P92）

In [10]:
def solve_A(k):
    iter_num = 0
    lower_bound = 2
    epsilon = 1e-12
    x_array = np.array([[10],[10],[10],[10],[10]])
    while sum(F_A(x_array, k) > epsilon * np.array([np.ones(5)]).T) :
        # 只要F0中有一个元素超过epsilon,则继续迭代
        iter_num += 1
        h = get_h(x_array, F_A(x_array, k), lower_bound)
        s = np.dot(np.linalg.inv(J(x_array, h, k)), F_A(x_array, k))
        a = 1
        x_array_ori = x_array
        x_array = x_array_ori - a * s
        while sum(x_array >= lower_bound * np.array([np.ones(5)]).T) < 5:
            a = a * 0.5
            x_array = x_array_ori - a * s
        
    print('Return after %d iterations' % iter_num)
    return x_array

In [11]:
print(solve_A(119))

Return after 3 iterations
[[ 6.37442356]
 [ 7.85596308]
 [ 9.76818647]
 [14.9977638 ]
 [22.83151734]]
